<a href="https://colab.research.google.com/github/Geoff24/Show-Renewal-Prediction/blob/main/TV_Shows_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd

netflix_tv = pd.read_csv("https://raw.githubusercontent.com/Geoff24/Show-Renewal-Prediction/main/netflix_titles.csv").drop(["director", "cast", "country", "date_added", "release_year", "rating", "listed_in", "show_id", "description"], axis = 1)
netflix_tv = netflix_tv[netflix_tv.type == "TV Show"].drop(["type"], axis = 1)
netflix_tv

,title,duration
1,Blood & Water,2 Seasons
2,Ganglands,1 Season
3,Jailbirds New Orleans,1 Season
4,Kota Factory,2 Seasons
5,Midnight Mass,1 Season
...,...,...
8795,Yu-Gi-Oh! Arc-V,2 Seasons
8796,Yunus Emre,2 Seasons
8797,Zak Storm,3 Seasons
8800,Zindagi Gulzar Hai,1 Season


In [55]:
tv_shows = pd.read_csv("https://raw.githubusercontent.com/Geoff24/Show-Renewal-Prediction/main/tv_shows.csv").drop(["Unnamed: 0", "ID", "Year", "Age", "Hulu", "Prime Video", "Disney+", "Type"], axis = 1)
tv_shows = tv_shows[tv_shows.Netflix == 1].drop(["Netflix"], axis = 1).rename({"Title": "title"}, axis =1)
tv_shows

,title,IMDb,Rotten Tomatoes
0,Breaking Bad,9.4/10,100/100
1,Stranger Things,8.7/10,96/100
2,Attack on Titan,9.0/10,95/100
3,Better Call Saul,8.8/10,94/100
4,Dark,8.8/10,93/100
...,...,...,...
4701,Human Nature,NaN,13/100
5064,The Bad Batch,8.2/10,82/100
5097,The Muppets,7.4/10,64/100
5231,Pick of the Litter,8.1/10,47/100


In [63]:
netflix_tv_shows = pd.merge(netflix_tv, tv_shows, on = "title").dropna()

netflix_tv_shows["IMDb"] = netflix_tv_shows["IMDb"].str.replace("/10", "").astype("float")
netflix_tv_shows["Rotten Tomatoes"] = netflix_tv_shows["Rotten Tomatoes"].str.replace("/100", "").astype("int")
netflix_tv_shows["duration"] = netflix_tv_shows["duration"].str.replace(" Season", "").str.replace("s", "").astype("int")

netflix_tv_shows["Multiple Seasons"] = (netflix_tv_shows.duration>1).astype("int")

netflix_tv_shows

,title,duration,IMDb,Rotten Tomatoes,Multiple Seasons
0,Blood & Water,2,6.4,56,1
1,Dear White People,4,6.2,66,1
2,Resurrection: Ertugrul,5,8.0,73,1
3,Love on the Spectrum,2,8.5,59,1
4,Sex Education,3,8.3,84,1
...,...,...,...,...,...
1629,The Jungle Book,1,4.2,39,0
1630,The Rap Game,1,5.7,43,0
1631,The Universe,1,8.7,65,0
1632,The Year of Happiness and Love,1,5.2,29,0


In [64]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

x = netflix_tv_shows[["Rotten Tomatoes", "IMDb"]]
y = netflix_tv_shows['Multiple Seasons']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

In [65]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score

tree = DecisionTreeClassifier(criterion='gini')
tree.fit(x_train, y_train)

yp = tree.predict(x_test)
print(accuracy_score(y_test, yp))

0.6038338658146964


In [69]:
from sklearn.model_selection import GridSearchCV
import pandas as pd

search_space = {
    "min_samples_leaf" : [1, 5, 10],
    "max_depth" : [1, 3, 4] 
}

gridSearch = GridSearchCV(tree, search_space)
gridSearch.fit(x_train, y_train)

print(pd.concat([pd.DataFrame(gridSearch.cv_results_["params"]), pd.DataFrame(gridSearch.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1), "\n")
print(gridSearch.best_params_)
print(gridSearch.best_score_)

   max_depth  min_samples_leaf  Accuracy
0          1                 1  0.691476
1          1                 5  0.691476
2          1                10  0.691476
3          3                 1  0.683470
4          3                 5  0.682676
5          3                10  0.687480
6          4                 1  0.684276
7          4                 5  0.677067
8          4                10  0.677070 

{'max_depth': 1, 'min_samples_leaf': 1}
0.6914763052208837
